<a href="https://colab.research.google.com/github/Maks3174/Phy/blob/main/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%BD%D0%B0_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Завдання

* Створіть Reader
* Створіть датасет та розділіть його на тренувальні та тестові дані
* Виберіть метрики для поріняння якості моделей
* На основі метрик виберіть найкращу модель
* Використайте `optuna` для підбору найкращих параметрів



In [1]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module7/data/Blog%20Ratings.csv")

df.head()

,blog_id,userId,ratings
0,9025,11,3.5
1,9320,11,5.0
2,9246,11,3.5
3,9431,11,5.0
4,875,11,2.0


In [8]:
!pip install scikit-surprise
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.9 MB/s eta 0:00:00


In [9]:
import pandas as pd
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split
from surprise import SVD, KNNBasic, BaselineOnly
from surprise import accuracy
from surprise.model_selection import cross_validate
import optuna
from surprise import SVD

In [10]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['userId', 'blog_id', 'ratings']], reader)

In [11]:
trainset, testset = train_test_split(data, test_size=0.2)

In [12]:
models = {
    'SVD': SVD(),
    'KNNBasic': KNNBasic(),
    'BaselineOnly': BaselineOnly()
}

for model_name, model in models.items():
    model.fit(trainset)
    predictions = model.test(testset)
    rmse = accuracy.rmse(predictions, verbose=False)
    mae = accuracy.mae(predictions, verbose=False)
    print(f'{model_name} - RMSE: {rmse}, MAE: {mae}')

SVD - RMSE: 1.8335155377637702, MAE: 1.6295993339349502
Computing the msd similarity matrix...
Done computing similarity matrix.
KNNBasic - RMSE: 1.9084778516468108, MAE: 1.6620478286290084
Estimating biases using als...
BaselineOnly - RMSE: 1.7949719026698192, MAE: 1.615912703126898


In [13]:
def objective(trial):
    n_factors = trial.suggest_int('n_factors', 20, 200)
    n_epochs = trial.suggest_int('n_epochs', 10, 50)
    lr_all = trial.suggest_float('lr_all', 0.001, 0.01)
    reg_all = trial.suggest_float('reg_all', 0.02, 0.2)

    model = SVD(n_factors=n_factors, n_epochs=n_epochs, lr_all=lr_all, reg_all=reg_all)
    model.fit(trainset)
    predictions = model.test(testset)
    rmse = accuracy.rmse(predictions, verbose=False)

    return rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

best_params = study.best_params
print("Найкращі параметри:", best_params)

best_model = SVD(**best_params)
best_model.fit(trainset)
predictions = best_model.test(testset)
final_rmse = accuracy.rmse(predictions)

[I 2024-09-01 08:49:54,646] A new study created in memory with name: no-name-cc1a5a9c-d189-4930-8071-be2ba3929613
[I 2024-09-01 08:49:56,798] Trial 0 finished with value: 1.774277701091629 and parameters: {'n_factors': 58, 'n_epochs': 15, 'lr_all': 0.001025904032628559, 'reg_all': 0.18578516189938846}. Best is trial 0 with value: 1.774277701091629.
[I 2024-09-01 08:50:04,916] Trial 1 finished with value: 1.8315852702829365 and parameters: {'n_factors': 165, 'n_epochs': 28, 'lr_all': 0.008571598531038256, 'reg_all': 0.1482463010640194}. Best is trial 0 with value: 1.774277701091629.
[I 2024-09-01 08:50:07,922] Trial 2 finished with value: 1.7856936875187963 and parameters: {'n_factors': 74, 'n_epochs': 19, 'lr_all': 0.0018079122065383243, 'reg_all': 0.06591950483860037}. Best is trial 0 with value: 1.774277701091629.
[I 2024-09-01 08:50:18,384] Trial 3 finished with value: 1.8848606503056722 and parameters: {'n_factors': 147, 'n_epochs': 48, 'lr_all': 0.008892118783556838, 'reg_all': 0.

Найкращі параметри: {'n_factors': 58, 'n_epochs': 15, 'lr_all': 0.001025904032628559, 'reg_all': 0.18578516189938846}
RMSE: 1.7743
